In [ ]:
!pip install pymorphy2-dicts-ru
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e1b8695ab5665c1e3e7cd124ea4cc2d428a6b241085080f741a166765a752f17
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 467.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.8 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np
from collections import Counter
import string
from tqdm import tqdm
import pymorphy2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/medical_data.csv'

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv(path)
data.head()

,MedicalRecordKey,MedicalRecordDate,PatientKey,Возраст,Пол,СпециальностьВрача,Жалобы,ПеренесенныеЗаболевания,ПеренесенныеОперации,ПринимаемыеПрепараты
0,106627476,2023-09-01 18:39:46.000,0xA52FAC0660E364A3BD2338AD6749D320FF3E08338230...,9764,1.0,Врач акушер гинеколог,на мажущие выделения из половых путей,"Факт получения травм, ожогов, ран, проведения ...",Отрицает,Левотироксин 75 мкг/сут
1,106627477,2023-09-01 18:39:45.000,0xC03852BF5FDC992B86A87B1215BC47EBF66C1AB03F6D...,13814,1.0,Врач уролог,на дискомфорт при мочеиспускании на учащенное ...,"Факт получения травм, ожогов, ран, проведения ...",""" аппендэктомия 2013. кесарево сечение - пр...","монурал, канефрон."
2,106627482,2023-09-01 18:39:53.000,0x417FA75C8DAD480F952C6931B305A0218311A4BE61E2...,10311,0.0,Врач дерматовенеролог,На покраснение и шелушение кожи головы.,ГЭРБ ЖДА,-,Колекальльциферол - 4000 МЕ Калия йодид Мальто...
3,106627486,2023-09-01 18:39:57.000,0x922886B7EA4058AC1BA576BEC37C24A4A2E8AB65E912...,22669,1.0,Врач кардиолог,Подписано информированное добровольное согласи...,узловой зоб,"аппендицит в 15-летнем возрасте, удаление кист...","на пенициллиновый ряд - отек квинке, сульфанил..."
4,106627490,2023-09-01 18:40:03.000,0xFAB438E0CA9FFC5CFF28E0B33914E95FEE9524108639...,15586,1.0,Врач акушер гинеколог,Жалобы на болезненность и нагрубание правой мо...,"Факт получения травм, ожогов, ран, проведения ...",аппендэктомия в 5 лет Импланты молочных желе...,Не отягощен


In [ ]:
data = data.rename(columns={'Возраст': 'age',
                            'СпециальностьВрача': 'Doctor',
                            'Пол': 'Sex',
                            'Жалобы': 'Complaints',
                            'ПеренесенныеЗаболевания': 'illness',
                            'ПеренесенныеОперации': 'Operations',
                            'ПринимаемыеПрепараты': 'Preparation'})

In [ ]:
reports = data['Complaints'].str.lower().unique()

In [ ]:
# sents_reports = []
# for elem in reports:
#   temp = elem.split(',')
#   temp = [x.strip() for x in temp]
#   temp = sent_tokenize(''.join(temp))
#   sents_reports.extend(temp)

In [ ]:
def remove_alien_chars(text, chars):
    return " ".join([ch for ch in text if ch not in chars]).split()

In [87]:
def remove_stopwords(tokens):
  russian_stopwords = stopwords.words('russian') + ['править','это']
  return remove_alien_chars(tokens, russian_stopwords)

In [ ]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text, morph):
  res = [morph.parse(word)[0].normal_form for word in text]
  return res

In [102]:
words = []
sents = []
for text in tqdm(reports):
  temp =  word_tokenize(text)
  temp = remove_alien_chars(lemmatize(temp, morph), string.punctuation)
  temp = remove_stopwords(temp)
  words.extend(temp)
  sents.append(temp)

100%|██████████| 10291/10291 [01:14<00:00, 138.27it/s]


In [99]:
tokens = word_tokenize(reports[2500])
text = remove_alien_chars(lemmatize(tokens, morph), string.punctuation)
remove_stopwords(text)

['момент',
 'осмотр',
 'жалоба',
 'предъявлять',
 'узи',
 'омт',
 '31.08.2023',
 'миома',
 'тело',
 'матка']

In [ ]:
for i in range(words.count("''")):
  words.remove("''")

for i in range(words.count("``")):
  words.remove("``")

In [84]:
word_counts = Counter(words)
top_values = word_counts.most_common(10000)

In [86]:
len(top_values)

8681

In [ ]:
for elem in top_values[:100]:
  print(elem[0])

Отобрав все жалобы можно составить ТОП-10 на что чаще всего жалуются пациенты в выборке:

1. Боль
2. Дискомфорт
3. Слабость
4. Живот
5. Нос (Залложенность)
6. Горло
7. Кашель
8. Сустав
9. Выделения
10. Кожа


In [90]:
symptoms = [
    "боль",
    "дискомфорт",
    "слабость",
    "живот",
    "заложенность",
    "горло",
    "кашель",
    "сустав",
    "выделения",
    "кожа"
]

In [131]:
dictinary = dict()
id = 1
for word in symptoms:
  dictinary[word] = id
  id+=1


In [135]:
def vectorize_sequences(sequences, dictin):
  result = []
  for x in dictin:
    if x in sequences:
      result.append(1)
    else:
      result.append(0)
  return result



In [137]:
values = []

for text in tqdm(reports):
  tokens = word_tokenize(text)
  tokens = remove_alien_chars(lemmatize(tokens, morph), string.punctuation)
  tokens = remove_stopwords(tokens)

  vector = vectorize_sequences(tokens, symptoms)
  values.append(vector)




In [139]:
df = pd.DataFrame(values, columns = symptoms)
df.head()

,боль,дискомфорт,слабость,живот,заложенность,горло,кашель,сустав,выделения,кожа
0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


In [142]:
new_df = pd.concat([data, df], axis=1)
new_df.head()

,MedicalRecordKey,MedicalRecordDate,PatientKey,age,Sex,Doctor,Complaints,illness,Operations,Preparation,боль,дискомфорт,слабость,живот,заложенность,горло,кашель,сустав,выделения,кожа
0,106627476,2023-09-01 18:39:46.000,0xA52FAC0660E364A3BD2338AD6749D320FF3E08338230...,9764,1.0,Врач акушер гинеколог,на мажущие выделения из половых путей,"Факт получения травм, ожогов, ран, проведения ...",Отрицает,Левотироксин 75 мкг/сут,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,106627477,2023-09-01 18:39:45.000,0xC03852BF5FDC992B86A87B1215BC47EBF66C1AB03F6D...,13814,1.0,Врач уролог,на дискомфорт при мочеиспускании на учащенное ...,"Факт получения травм, ожогов, ран, проведения ...",""" аппендэктомия 2013. кесарево сечение - пр...","монурал, канефрон.",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,106627482,2023-09-01 18:39:53.000,0x417FA75C8DAD480F952C6931B305A0218311A4BE61E2...,10311,0.0,Врач дерматовенеролог,На покраснение и шелушение кожи головы.,ГЭРБ ЖДА,-,Колекальльциферол - 4000 МЕ Калия йодид Мальто...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,106627486,2023-09-01 18:39:57.000,0x922886B7EA4058AC1BA576BEC37C24A4A2E8AB65E912...,22669,1.0,Врач кардиолог,Подписано информированное добровольное согласи...,узловой зоб,"аппендицит в 15-летнем возрасте, удаление кист...","на пенициллиновый ряд - отек квинке, сульфанил...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,106627490,2023-09-01 18:40:03.000,0xFAB438E0CA9FFC5CFF28E0B33914E95FEE9524108639...,15586,1.0,Врач акушер гинеколог,Жалобы на болезненность и нагрубание правой мо...,"Факт получения травм, ожогов, ран, проведения ...",аппендэктомия в 5 лет Импланты молочных желе...,Не отягощен,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
new_df.to_csv('New_medical_data.csv')